# Introduction
This constellation of multiple Jupyter Notebooks documents the data mining process for a train data set. The selected data set contains records for various train journeys since november 2023. The data has been created by manually documenting key statistics about every journey in a spreadsheet. The aim of this analysis is to explore patterns and potential correlations within the data set. The data set includes information about the train line, departure and arrival stations as well as date & time, delay and ticket inspections. The exact attributes will be analyzed later on.
This data set has been chosen due to the personal connection based on manually creating the data over an extended period of time. Predictions and classifications could also help improve future decisions about which train connection to take for a minimal risk of delay.

The primary questions for this analysis are:
1. Is there a correlation between the train line and other journey statistics? This includes examining whether certain lines tend to have more delay or higher crowding.
2. Can journeys be classified based on key features? Using classification techniques it will be attempted to predict if a train is for example 'likely to be crowded'.
3. Is the likelihood of ticket inspections dependent on the time of day or month?

# Data exploration
The data set consists of two related spread sheets: one for the train journeys and general statistics about these journeys, the other for annoucements during the train drive.

## Train journeys

### Attributes
The data set has a limited amount of attributes which can be explained either by their name or by annotations in the data set. The original names are in german but for language consistency the english translation will be used when talking about the attributes. The most important attributes are:
- **ID** ("Lfd. Nummer") is an ID for every record
- **train line** ("Lininen Nummer") is the train number
- **departure station** ("Start Haltestelle") is the departure station
- **arrival station** ("Ziel Haltestelle") is the arrival station
- **planned departure** ("Planmäßige Abfahrtszeit") and date ("Datum") are the time and date of depature

These attributes are normally always known before the journey is even taken. They will be referenced as **fixed information** of a journey. The following attributes are determined during and after the journey has been taken and referenced as **variable information**:
- **delay** "Verspätung in min" the delay upon arrival in minutes, however delays are only documented if they are 2 min or longer. Arrival counts as the moment the first passengers step out of the vehicle.
- **ticket checked** ("Kontrolliert") if there was a ticket inspection. For context in german trains there are commonly no gates to force you to buy a ticket but random checks are performed by ticket inspectors in the trian.
- **platform changed** ("Gleis verlegt") if the platform (departure or arrival) was different from the planned platform
- **crowdiness** ("Fülle des Zuges") describes how full the train was
- **train model** ("Zugmodel") is only documented for specific trains which can be classified as old or new 
- **cleanliness** ("Sauberkeit") takes into consideration if there is trash, if floor or seats are dirty and any smell is in the train

Additionally there are three attributes which exist because the delay of one train does not always reflect the true experience of train driving. For example if train A has a delay of 15 min but due to that the connecting train B is missed, the traveler might have to wait an additional 15 min for the next train. Therefore the total delay would be 30 min.
- **relative planned departure** ("rel. planmäßige Abfahrtsyeit") ....
- **relative delay** ("rel. Verspätung") ....
- **alternative connection** ("Alternativer Anschluss") ...

The attribute note ("Bemerkung") is more a personal note and will not be taken into consideration for this analysis.


The following table shows the types of attributes:

|               Attribute Name | Attribute Type | Possible values |
|-----------------------------:|----------------|-----------------|
|                  Lfd. Nummer |                |                 |
|                Linien Nummer |                |                 |
|            Start Haltestelle |                |                 |
| Ziel Haltestelle             |                |                 |
| Planmäßige Abfahrtszeit      |                |                 |
| Verspätung in min            |                |                [1] |
| Kontrolliert                 |                |                 |
| Gleis verlegt                |                |                 |
| Fülle des Zuges              |                |         [2]        |
| Zugmodel                     |                |           [3]      |
| Sauberkeit                   |                |                 |
| rel. planmäßige Abfahrtszeit |                |                 |
| rel. Verspätung              |                |                 |
| Alternativer Anschluss       |                |                 |

[1] 

### Sample data